In [ ]:
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1MB 1.4MB/s eta 0:13:13tcmalloc: large alloc 1147494400 bytes == 0x55f180b00000 @  0x7f1fc796f615 0x55f146bec02c 0x55f146ccc17a 0x55f146beee4d 0x55f146ce0c0d 0x55f146c630d8 0x55f146c5dc35 0x55f146bf073a 0x55f146c62f40 0x55f146c5dc35 0x55f146bf073a 0x55f146c5f93b 0x55f146ce1a56 0x55f146c5efb3 0x55f146ce1a56 0x55f146c5efb3 0x55f146ce1a56 0x55f146c5efb3 0x55f146ce1a56 0x55f146d630d1 0x55f146cc42f9 0x55f146c2eac4 0x55f146bef8a9 0x55f146c63b0a 0x55f146bf065a 0x55f146c5eb0e 0x55f146c5e235 0x55f146bf073a 0x55f146c5eb0e 0x55f146bf065a 0x55f146c5eb0e
     |█████████████████               | 1055.7MB 1.3MB/s eta 0:12:03tcmalloc: large alloc 1434370048 bytes == 0x55f1c5156000 @  0x7f1fc796f615 0x55f146bec02c 0x55f146ccc17a 0x55f146beee4d 0x55f146ce0c0d 0x55f146c630d8 0x55f146c5dc35 0x55f146bf073a 0x55f146c62f40 0x55f146c5dc35 0x55f146bf073a 0x55f146c5f93b 0x55f146ce1a56 0x55f146c5efb3 0

In [ ]:
# !pip install torch==1.8.0

In [ ]:
!gdown --id 1Y8EOFLIRCcKpe_e0pO03yCAosTRjRMtC

Downloading...
From: https://drive.google.com/uc?id=1Y8EOFLIRCcKpe_e0pO03yCAosTRjRMtC
To: /content/UTKFace.zip
347MB [00:03, 109MB/s]


In [ ]:
!unzip -q /content/UTKFace.zip -d data

In [ ]:
import os

In [ ]:
n = len(os.listdir('/content/data/UTKFace'))
n

23708

In [ ]:
!ls

data  sample_data  UTKFace.zip


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt
import time
import os
import copy
import random

# Gender

In [ ]:
class AgeDataset(Dataset):
  def __init__(self, img_files, img_dir):
    self.img_dir = img_dir
    self.img_files = img_files
    self.image_transforms = transforms.Compose([
      transforms.Resize((224,224)),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

  def __len__(self):
    return len(self.img_files)

  def __getitem__(self, index):
    img_name = self.img_files[index]
    gender = int(img_name.split("_")[1])
    image = Image.open(os.path.join(self.img_dir, img_name))

    image = self.image_transforms(image)
    gender = torch.tensor(gender, dtype=torch.long)

    return image, gender

In [ ]:
img_dir = "data/UTKFace/"
train_val_ratio = 0.3

img_filenames = os.listdir(img_dir)
random.shuffle(img_filenames)
split = int(train_val_ratio * len(img_filenames))

train_files = img_filenames[split:]
val_files = img_filenames[:split]
print(len(train_files), len(val_files))

train_set = AgeDataset(train_files, img_dir)
val_set = AgeDataset(val_files, img_dir)

train_dataloader = DataLoader(train_set, batch_size=128, shuffle=True)
val_dataloader = DataLoader(val_set, batch_size=128, shuffle=False)

16596 7112


In [ ]:
for img, label in train_dataloader:
  print(img, label)
  break

tensor([[[[-1.5528, -1.6555, -1.7412,  ..., -1.9980, -1.9809, -1.9638],
          [-1.5528, -1.6727, -1.7583,  ..., -1.9809, -1.9638, -1.9638],
          [-1.5528, -1.6727, -1.7583,  ..., -1.9638, -1.9467, -1.9467],
          ...,
          [-1.7412, -1.7412, -1.7583,  ..., -0.5596, -0.7137, -0.8164],
          [-1.8782, -1.8782, -1.8953,  ..., -0.5424, -0.7137, -0.8164],
          [-1.9638, -1.9638, -1.9809,  ..., -0.5424, -0.7137, -0.8164]],

         [[-1.3529, -1.4580, -1.5455,  ..., -1.7381, -1.7206, -1.7031],
          [-1.3529, -1.4755, -1.5630,  ..., -1.7206, -1.7031, -1.7031],
          [-1.3529, -1.4755, -1.5630,  ..., -1.7031, -1.6856, -1.6856],
          ...,
          [-1.6155, -1.6155, -1.6331,  ..., -1.0553, -1.1954, -1.2654],
          [-1.7556, -1.7556, -1.7731,  ..., -1.0553, -1.1954, -1.2654],
          [-1.8431, -1.8431, -1.8606,  ..., -1.0553, -1.1954, -1.2654]],

         [[-1.2990, -1.4036, -1.4907,  ..., -1.4907, -1.4733, -1.4559],
          [-1.2990, -1.4210, -

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
model = torch.hub.load('pytorch/vision:v0.9.0', 'shufflenet_v2_x1_0', pretrained=True)
for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

model = model.to(device)

dataloaders = {'train': train_dataloader, 'val': val_dataloader}
dataset_sizes = {'train': len(train_files), 'val': len(val_files)}

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


In [ ]:
# warm up

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
# optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.001)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

model = train_model(model, criterion, optimizer, exp_lr_scheduler,
                       num_epochs=1)

Epoch 0/0
----------
train Loss: 0.6553 Acc: 0.6552
val Loss: 0.6207 Acc: 0.7236

Training complete in 1m 18s
Best val Acc: 0.723566


In [ ]:
# fine-tuning
for param in model.parameters():
    param.requires_grad = True

optimizer = optim.Adam(model.parameters(), lr=0.0003)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

model = train_model(model, criterion, optimizer, exp_lr_scheduler,
                       num_epochs=10)

Epoch 0/9
----------
train Loss: 0.3227 Acc: 0.8631
val Loss: 0.2212 Acc: 0.9075

Epoch 1/9
----------
train Loss: 0.1728 Acc: 0.9299
val Loss: 0.2137 Acc: 0.9106

Epoch 2/9
----------
train Loss: 0.1050 Acc: 0.9600
val Loss: 0.2252 Acc: 0.9161

Epoch 3/9
----------
train Loss: 0.0622 Acc: 0.9767
val Loss: 0.3047 Acc: 0.9052

Epoch 4/9
----------
train Loss: 0.0409 Acc: 0.9854
val Loss: 0.2827 Acc: 0.9163

Epoch 5/9
----------
train Loss: 0.0312 Acc: 0.9881
val Loss: 0.3372 Acc: 0.9106

Epoch 6/9
----------
train Loss: 0.0216 Acc: 0.9917
val Loss: 0.3630 Acc: 0.9116

Epoch 7/9
----------
train Loss: 0.0119 Acc: 0.9960
val Loss: 0.3628 Acc: 0.9144

Epoch 8/9
----------
train Loss: 0.0073 Acc: 0.9976
val Loss: 0.3567 Acc: 0.9145

Epoch 9/9
----------
train Loss: 0.0068 Acc: 0.9976
val Loss: 0.3837 Acc: 0.9145

Training complete in 15m 32s
Best val Acc: 0.916339


## save model

In [ ]:
os.mkdir("./gender_shufflenetv2")
model_path = "./gender_shufflenetv2/model.pt"
torch.save(model.state_dict(), model_path)

## load and inference

In [ ]:
def eval_model(model, criterion):
    since = time.time()

    for epoch in range(1):

        # Each epoch has a training and validation phase
        for phase in ['val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

        print()

    time_elapsed = time.time() - since
    print('Val complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

In [ ]:
new_model = torch.hub.load('pytorch/vision:v0.9.0', 'shufflenet_v2_x1_0', pretrained=True)

num_ftrs = new_model.fc.in_features
new_model.fc = nn.Linear(num_ftrs, 2)

new_model.load_state_dict(torch.load(model_path))

new_model = new_model.to(device)
criterion = nn.CrossEntropyLoss()

eval_model(new_model, criterion)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


val Loss: 0.2827 Acc: 0.9163

Val complete in 0m 23s


# Age

In [ ]:
class AgeDataset(Dataset):
  def __init__(self, img_files, img_dir):
    self.img_dir = img_dir
    self.img_files = img_files
    self.image_transforms = transforms.Compose([
      transforms.Resize((224,224)),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

  def __len__(self):
    return len(self.img_files)

  def __getitem__(self, index):
    img_name = self.img_files[index]
    age = float(img_name.split("_")[0]) / 116
    image = Image.open(os.path.join(self.img_dir, img_name))

    image = self.image_transforms(image)

    return image, age

In [ ]:
img_dir = "data/UTKFace/"
train_val_ratio = 0.3

img_filenames = os.listdir(img_dir)
random.shuffle(img_filenames)
split = int(train_val_ratio * len(img_filenames))

train_files = img_filenames[split:]
val_files = img_filenames[:split]
print(len(train_files), len(val_files))

train_set = AgeDataset(train_files, img_dir)
val_set = AgeDataset(val_files, img_dir)

train_dataloader = DataLoader(train_set, batch_size=128, shuffle=True)
val_dataloader = DataLoader(val_set, batch_size=128, shuffle=False)

16596 7112


In [ ]:
for img, label in train_dataloader:
  print(img, label)
  break

tensor([[[[-1.0733, -1.2788, -1.5357,  ..., -1.8953, -1.8782, -1.8610],
          [-1.1075, -1.3130, -1.5528,  ..., -1.8782, -1.8782, -1.8610],
          [-1.1418, -1.3473, -1.5528,  ..., -1.8610, -1.8610, -1.8439],
          ...,
          [-0.6623, -0.6452, -0.6109,  ..., -1.8268, -1.8439, -1.8610],
          [-0.6623, -0.6281, -0.5767,  ..., -1.8268, -1.8439, -1.8782],
          [-0.6452, -0.5938, -0.5424,  ..., -1.8439, -1.8610, -1.8953]],

         [[-1.2129, -1.4230, -1.6856,  ..., -1.9132, -1.8957, -1.8782],
          [-1.2479, -1.4580, -1.7031,  ..., -1.8957, -1.8957, -1.8782],
          [-1.2829, -1.4930, -1.7031,  ..., -1.8782, -1.8782, -1.8606],
          ...,
          [-0.9678, -0.9503, -0.9153,  ..., -1.8256, -1.8431, -1.8782],
          [-0.9678, -0.9328, -0.8803,  ..., -1.8256, -1.8431, -1.8957],
          [-0.9503, -0.8978, -0.8452,  ..., -1.8256, -1.8431, -1.8782]],

         [[-1.1770, -1.3513, -1.6127,  ..., -1.6824, -1.6650, -1.6476],
          [-1.2119, -1.3861, -

In [ ]:
# import torchvision

# def imshow(inp, title=None):
#     """Imshow for Tensor."""
#     inp = inp.numpy().transpose((1, 2, 0))
#     mean = np.array([0.485, 0.456, 0.406])
#     std = np.array([0.229, 0.224, 0.225])
#     inp = std * inp + mean
#     inp = np.clip(inp, 0, 1)
#     plt.imshow(inp)
#     if title is not None:
#         plt.title(title)
#     plt.pause(0.001)  # pause a bit so that plots are updated


# # Get a batch of training data
# inputs, classes = next(iter(train_dataloader))

# # Make a grid from batch
# out = torchvision.utils.make_grid(inputs)

# imshow(out, title=[x.item() for x in classes])

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_mae = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            # epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Mae: {:.4f}'.format(
                phase, epoch_loss, epoch_loss))

            # deep copy the model
            if phase == 'val' and epoch_loss < best_mae:
                best_mae = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Mae: {:4f}'.format(best_mae))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
model = torch.hub.load('pytorch/vision:v0.9.0', 'shufflenet_v2_x1_0', pretrained=True)
for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)

model = model.to(device)

dataloaders = {'train': train_dataloader, 'val': val_dataloader}
dataset_sizes = {'train': len(train_files), 'val': len(val_files)}

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


In [ ]:
# warm up

criterion = nn.L1Loss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
# optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.001)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

model = train_model(model, criterion, optimizer, exp_lr_scheduler,
                       num_epochs=1)

Epoch 0/0
----------


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([84])) that is different to the input size (torch.Size([84, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


train Loss: 0.1389 Mae: 0.1389
val Loss: 0.1295 Mae: 0.1295

Training complete in 1m 27s
Best val Mae: 0.129487


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([72])) that is different to the input size (torch.Size([72, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


In [ ]:
# fine-tuning
for param in model.parameters():
    param.requires_grad = True

# optimizer = optim.SGD(model.parameters(), lr=0.0003, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.0003)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

model = train_model(model, criterion, optimizer, exp_lr_scheduler,
                       num_epochs=10)

Epoch 0/9
----------


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([84])) that is different to the input size (torch.Size([84, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


train Loss: 0.1305 Mae: 0.1305


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([72])) that is different to the input size (torch.Size([72, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


val Loss: 0.1283 Mae: 0.1283

Epoch 1/9
----------
train Loss: 0.1301 Mae: 0.1301
val Loss: 0.1274 Mae: 0.1274

Epoch 2/9
----------
train Loss: 0.1302 Mae: 0.1302
val Loss: 0.1275 Mae: 0.1275

Epoch 3/9
----------
train Loss: 0.1301 Mae: 0.1301
val Loss: 0.1274 Mae: 0.1274

Epoch 4/9
----------
train Loss: 0.1301 Mae: 0.1301
val Loss: 0.1274 Mae: 0.1274

Epoch 5/9
----------
train Loss: 0.1301 Mae: 0.1301
val Loss: 0.1274 Mae: 0.1274

Epoch 6/9
----------
train Loss: 0.1301 Mae: 0.1301
val Loss: 0.1274 Mae: 0.1274

Epoch 7/9
----------
train Loss: 0.1299 Mae: 0.1299
val Loss: 0.1274 Mae: 0.1274

Epoch 8/9
----------
train Loss: 0.1299 Mae: 0.1299
val Loss: 0.1274 Mae: 0.1274

Epoch 9/9
----------
train Loss: 0.1299 Mae: 0.1299
val Loss: 0.1274 Mae: 0.1274

Training complete in 17m 5s
Best val Mae: 0.128278


## save model

In [ ]:
os.mkdir("./age_shufflenetv2")
model_path = "./age_shufflenetv2/model.pt"
torch.save(model.state_dict(), model_path)

## load and inference

In [ ]:
def eval_model(model, criterion):
    since = time.time()

    # best_model_wts = copy.deepcopy(model.state_dict())
    # best_mae = 0.0

    for epoch in range(1):
        # print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        # print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                # statistics
                running_loss += loss.item() * inputs.size(0)

            epoch_loss = running_loss / dataset_sizes[phase]

            print('{} Loss: {:.4f} Mae: {:.4f}'.format(
                phase, epoch_loss, epoch_loss))

        print()

    time_elapsed = time.time() - since
    print('Val complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

In [ ]:
new_model = torch.hub.load('pytorch/vision:v0.9.0', 'shufflenet_v2_x1_0', pretrained=True)

num_ftrs = new_model.fc.in_features
new_model.fc = nn.Linear(num_ftrs, 1)

new_model.load_state_dict(torch.load(model_path))

new_model = new_model.to(device)
criterion = nn.L1Loss()

eval_model(new_model, criterion)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


val Loss: 0.1283 Mae: 0.1283

Val complete in 0m 26s


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([72])) that is different to the input size (torch.Size([72, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


# download the model file

In [ ]:
!pwd

/content


In [ ]:
!ls

age_shufflenetv2  data	drive  gender_shufflenetv2  sample_data  UTKFace.zip


In [ ]:
!cp -R "./age_shufflenetv2" "/content/drive/MyDrive/Colab Notebooks/"

In [ ]:
!cp -R "./gender_shufflenetv2" "/content/drive/MyDrive/Colab Notebooks/"

In [ ]:
!ls

age_shufflenetv2  data	drive  gender_shufflenetv2  sample_data  UTKFace.zip


In [ ]:
!gdown --id "1nXHFCYHm9bQ3shB_4INU5c_VuMPckPZh"

Permission denied: https://drive.google.com/uc?id=1nXHFCYHm9bQ3shB_4INU5c_VuMPckPZh
Maybe you need to change permission over 'Anyone with the link'?


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/data

/content/drive/.shortcut-targets-by-id/1nXHFCYHm9bQ3shB_4INU5c_VuMPckPZh/data


In [ ]:
%cd ..

/content/drive/.shortcut-targets-by-id/1nXHFCYHm9bQ3shB_4INU5c_VuMPckPZh


In [ ]:
%cd /content

/content


In [ ]:
!ls

age_shufflenetv2  data	drive  gender_shufflenetv2  sample_data  UTKFace.zip


In [ ]:
TEST_DIR = "/content/drive/.shortcut-targets-by-id/1nXHFCYHm9bQ3shB_4INU5c_VuMPckPZh/data/"

## test age

In [ ]:
class AgeDataset(Dataset):
  def __init__(self, img_files, img_dir):
    self.img_dir = img_dir
    self.img_files = img_files
    self.image_transforms = transforms.Compose([
      transforms.Resize((224,224)),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

  def __len__(self):
    return len(self.img_files)

  def __getitem__(self, index):
    img_name = self.img_files[index]
    age = float(img_name.split("_")[0]) / 116
    image = Image.open(os.path.join(self.img_dir, img_name))

    image = self.image_transforms(image)

    return image, age

In [ ]:
img_dir = TEST_DIR
# train_val_ratio = 0.3

img_filenames = os.listdir(img_dir)
# random.shuffle(img_filenames)
# split = int(train_val_ratio * len(img_filenames))

val_files = img_filenames
print(len(val_files))

val_set = AgeDataset(val_files, img_dir)

val_dataloader = DataLoader(val_set, batch_size=64, shuffle=False)

61


In [ ]:
dataloaders = {'val': val_dataloader}
dataset_sizes = {'val': len(val_files)}

def eval_model(model, criterion):
    since = time.time()

    # best_model_wts = copy.deepcopy(model.state_dict())
    # best_mae = 0.0

    for epoch in range(1):
        # print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        # print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                # statistics
                running_loss += loss.item() * inputs.size(0)

            epoch_loss = running_loss / dataset_sizes[phase]

            print('{} Loss: {:.4f} Mae: {:.4f}'.format(
                phase, epoch_loss, epoch_loss))

        print()

    time_elapsed = time.time() - since
    print('Val complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))


new_model = torch.hub.load('pytorch/vision:v0.9.0', 'shufflenet_v2_x1_0', pretrained=True)

num_ftrs = new_model.fc.in_features
new_model.fc = nn.Linear(num_ftrs, 1)

model_path = "./age_shufflenetv2/model.pt"
new_model.load_state_dict(torch.load(model_path))

new_model = new_model.to(device)
criterion = nn.L1Loss()

eval_model(new_model, criterion)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


val Loss: 0.1101 Mae: 0.1101

Val complete in 0m 28s


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([61])) that is different to the input size (torch.Size([61, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


## test gender

In [ ]:
class AgeDataset(Dataset):
  def __init__(self, img_files, img_dir):
    self.img_dir = img_dir
    self.img_files = img_files
    self.image_transforms = transforms.Compose([
      transforms.Resize((224,224)),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

  def __len__(self):
    return len(self.img_files)

  def __getitem__(self, index):
    img_name = self.img_files[index]
    gender = int(img_name.split("_")[1])
    image = Image.open(os.path.join(self.img_dir, img_name))

    image = self.image_transforms(image)
    gender = torch.tensor(gender, dtype=torch.long)

    return image, gender

In [ ]:
img_dir = TEST_DIR
img_filenames = os.listdir(img_dir)

val_files = img_filenames
print(len(val_files))

val_set = AgeDataset(val_files, img_dir)

val_dataloader = DataLoader(val_set, batch_size=64, shuffle=False)

61


In [ ]:
dataloaders = {'val': val_dataloader}
dataset_sizes = {'val': len(val_files)}

def eval_model(model, criterion):
    since = time.time()

    for epoch in range(1):

        # Each epoch has a training and validation phase
        for phase in ['val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

        print()

    time_elapsed = time.time() - since
    print('Val complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    
new_model = torch.hub.load('pytorch/vision:v0.9.0', 'shufflenet_v2_x1_0', pretrained=True)

num_ftrs = new_model.fc.in_features
new_model.fc = nn.Linear(num_ftrs, 2)

model_path = "./gender_shufflenetv2/model.pt"
new_model.load_state_dict(torch.load(model_path))

new_model = new_model.to(device)
criterion = nn.CrossEntropyLoss()

eval_model(new_model, criterion)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


val Loss: 0.1811 Acc: 0.9344

Val complete in 0m 0s
